In [ ]:
# !pip install simplemma

In [ ]:
from simplemma import text_lemmatizer
import pandas as pd
import numpy as np
import ast
import re

Veri seti ön işleme


In [ ]:
# stop word ayıklaması yapmayan metin temizleme fonk.
def clean_text(text):
    text = text.replace("Â", "a")
    text = text.replace("â", "a")
    text = text.replace("î", "i")
    text = text.replace("Î", "ı")
    text = text.replace("İ", "i")
    text = text.replace("I", "ı")
    text = text.replace(u"\u00A0", " ")
    text = text.replace("|", " ")

    text = re.sub(r"@[A-Za-z0-9]+", " ", text)
    text = re.sub(r"(.)\1+", r"\1\1", text)
    text = re.sub(r"https?:\/\/\S+", " ", text)
    text = re.sub(r"http?:\/\/\S+", " ", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"#(\w+)", " ", text)
    text = re.sub(r"^\x00-\x7F]+", " ", text)
    text = re.sub(r"[^A-Za-zâîığüşöçİĞÜŞÖÇ]+", " ", text)
    text = re.sub(r"((https://[^\s]+))", " ", text)
    text = " ".join(text.lower().strip().split())

    return " ".join(text_lemmatizer(text, lang="tr"))

In [ ]:
clean_text("Hey! Naber???İyilik, senden... Ben de iyiyim.Çocuk filmi Kirpi Sonic filminin devam halkası olan yapımda, insan biçiminde bir kirpi olan Sonic'in atıldığı maceralar konu ediliyor.")

In [ ]:
df = pd.read_excel("filmveriseti.xlsx")
df.head()

In [ ]:
df["tur_list"] = df["tur"].apply(ast.literal_eval)
df.head()

In [ ]:
def clean_row(row):
    ozet = clean_text(row["ozet ve detaylar"])
    tur = clean_text(" ".join(row["tur_list"]))
    return ozet + " " + tur

In [ ]:
clean_row(df.iloc[1])

In [ ]:
df["clean"] = df.apply(lambda row: clean_row(row), axis=1)
df.head()

In [ ]:
filmler = df["clean"].values
print(filmler[:3])

Sözlük (Vocab) oluşturma


In [ ]:
word2idx = {}
i = 0

for film in filmler:
    for sozcuk in film.split():
        if sozcuk not in word2idx:
            word2idx[sozcuk] = i
            i += 1

In [ ]:
word2idx

TF matrisi oluşturma


In [ ]:
film_sayisi = len(filmler)
sozluk_buyuklugu = len(word2idx)

print(film_sayisi, sozluk_buyuklugu)

In [ ]:
tf = np.zeros((film_sayisi, sozluk_buyuklugu))

for i, film in enumerate(filmler):
    for sozcuk in film.split():
        tf[i, word2idx[sozcuk]] += 1 # sözcük sayma

    tf[i] = tf[i] / len(film.split()) # normalizasyon

print(tf[0])

IDF vektörü oluşturma


In [ ]:
# Örnek
toy_tf = np.array([
    [0.5, 0.0, 0.0, 0.5],
    [0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.0, 0.9],
    [0.3, 0.0, 0.1, 0.6],
    [0.7, 0.0, 0.0, 0.3],
])

toy_doc_freq = np.sum(toy_tf > 0, axis=0)
toy_doc_freq

In [ ]:
toy_idf = 5 / toy_doc_freq
print(toy_idf)
toy_idf = np.log(toy_idf)
print(toy_idf)

In [ ]:
# gerçek hesaplama kısmı
doc_freq = np.sum(tf > 0, axis=0)
print(doc_freq)

idf = np.log(film_sayisi / doc_freq)
print(idf)

TF-IDF matrisi oluşturma


In [ ]:
tf_idf = tf * idf

Benzerlik hesaplama ve sorgular


In [ ]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
def film2str(row_index):
    film = df.iloc[row_index]
    adi = film["film adi"]
    turu = ", ".join(film["tur_list"])
    ozet = re.sub(r"\n+", "\n", film["ozet ve detaylar"]).strip()
    if len(ozet) > 150:
        ozet = ozet[:150] + "..."

    return f"Film adı: {adi} \nTürü: {turu} \nÖzeti: {ozet}"


print(film2str(3))

In [ ]:
def benzerlerini_bul(film_adi, benzer_film_sayisi):
    film_indexi = df[df["film adi"] == film_adi].index[0]
    print("Aranan Film:")
    print(film2str(film_indexi))
    print()
    print("* " * 30)
    print()

    benzerlikler = []
    sorgu_film = tf_idf[film_indexi]
    for vec in tf_idf:
        benzerlik = cos_sim(sorgu_film, vec)
        benzerlikler.append(benzerlik)

    en_benzer_film_indexleri = (- np.array(benzerlikler)
                                ).argsort()[:benzer_film_sayisi + 1]
    print("Benzer Filmler:\n")
    for i, film_idx in enumerate(en_benzer_film_indexleri[1:]):
        print(str(i+1) + ") " + film2str(film_idx))
        print()
        print("- " * 20)
        print()

In [ ]:
benzerlerini_bul("Kirpi Sonic 2", 3)